# Warsztaty Python w Data Science

## Z Wikipedii do Pandasa
## Web Scraping 1 z 3
---
- ### Budowa web crawlera
  - #### Anatomia pająka
  - #### Zarządzanie granicą (*Crawling Frontier*)
  - #### Jak scrapować etycznie i bezpiecznie
- ### Anatomia strony WWW
  - #### HTML jaki jest - każdy widzi
  - #### Parsowanie pobranych danych
- ### Prosty, praktyczny scraper
- ### Z Wikipedii do Pandasa
---


## Budowa web crawlera

*Web crawler, webbot, pająk, spider, pełzacz, web wanderer, scraper, crawler* - program zbierający informacje o strukturze i treściach stron WWW. 

### Anatomia pająka


### <span style="color: cyan">*Pająk*</span> - to program który:
- odwiedza linki ze listy określanej jako <span style="color: cyan">granica (*the frontier*)</span>
- z odwiedzonych stron wyciąga informację:
  - w szczególności <span style="color: cyan">dalsze linki (web indexing)</span>
  - odpowiednie linki uzupełniają *granicę* crawlingu
  - zapisuje informację w sposób trwały (web crawler)


### Zarządzanie granicą (*Crawling Frontier*)

- Zakres granicy powinien być <span style="color: cyan">ZAWSZE</span> na początku określony
    - najlepiej z góry zawężony do określonej liczby i typu linków
- <span style="color: cyan">Granica winna być mocno ograniczana</span>
- <span style="color: cyan">Granica rozbudowywana powinna być BARDZO selektywnie</span>
- Duża granica powoduje problemy skali

---
### Jak scrapować etycznie i bezpiecznie:

1. Po pierwsze - nie szkodzić! Nie obciążaj niepotrzebnie strony scrapowanej
2. Przestrzegaj `robots.txt` i warunków korzystania z usługi
3. Miej na uwadze, że bazy danych są chronione na podstawie przepisów ustawy o ochronie baz danych 
4. Przestrzegaj RODO
5. Nie ukrywaj się
6. Gdzie to możliwe,  korzystaj z <span style="color: cyan">__API__</span>
---

# art. 8 ustawy z dnia 27 lipca 2001 r. o ochronie baz danych (Dz.U. Nr 28, poz. 1402 ze zm.) 

- #### 1. Wolno korzystać z istotnej, co do jakości lub ilości, części rozpowszechnionej bazy danych:

  - 1)   do własnego użytku osobistego, ale tylko z zawartości nieelektronicznej bazy danych,

  - 2)   w charakterze ilustracji, w celach _**dydaktycznych lub badawczych**_ (podkreslenie moje), ze wskazaniem źródła, jeżeli takie korzystanie jest uzasadnione niekomercyjnym celem, dla którego wykorzystano bazę,
  
  - 3)   do celów bezpieczeństwa wewnętrznego, postępowania sądowego lub administracyjnego.

- #### 2. Nie jest dozwolone powtarzające się i systematyczne pobieranie lub wtórne wykorzystanie sprzeczne z normalnym korzystaniem i powodujące nieusprawiedliwione naruszenie słusznych interesów producenta.
---

### Anatomia strony WWW
#### HTML jaki jest - każdy widzi


In [ ]:
html_doc = """
<html>
<head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
    <a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
    <a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
    <a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
    and they lived at the bottom of a well.</p>

<p class="story">...</p>
</body></html>
"""

---
#### Parsowanie pobranych danych

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')

print(soup.prettify())

---
## HTML ma
- strukturę drzewiastą
- znaczniki się zagnieżdzają i mają mieć (w teorii):
  - `<a>` - początek
  - `</a>` - koniec
- pomiędzy początkiem a końcem są tzw. dzieci
- znaczniki nie mogą się "zazębiac" np. `<a><b></a></b>` (w teorii ...)
- znaczniki mają atrybuty - np. `<a href="linkdostrony">Tu jest link</a>
---

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_doc)

In [ ]:
soup.p

In [ ]:
soup.p['class']

In [ ]:
soup.a

### Szukanie po znaczniku

In [ ]:
soup.find_all('a')

### Szukanie po id

In [ ]:
soup.find(id="link3")

### Wyciąganie atrybutów

In [ ]:
[ link.get('href') for link in soup.find_all('a')]

### Wędrowanie po drzewie

In [ ]:
soup.a

In [ ]:
soup.a.find_next_sibling("a")

In [ ]:
soup.p

In [ ]:
soup.p.find_next_sibling("p")

In [ ]:
pn=soup.p.find_next_sibling("p")
children = pn.children

In [ ]:
children

In [ ]:
lista = [ x for x in children ]
lista

In [ ]:
lista[1].get('href')

In [ ]:
head_tag = soup.head
head_tag

In [ ]:
for child in head_tag.children:
    print(child)

In [ ]:
for child in head_tag.descendants:
    print(child)

In [ ]:
last_a_tag = soup.find("a", id="link3")
last_a_tag


In [ ]:
last_a_tag.next_sibling

In [ ]:
last_a_tag.next_element

In [ ]:
last_a_tag.parent

### Szukanie przy użyciu predykatu (funckji logicznej)

In [ ]:
def has_class_but_no_id(tag):
    return tag.has_attr('class') and not tag.has_attr('id')

soup.find_all(has_class_but_no_id)

In [ ]:
soup.find_all(id='link2')

In [ ]:
soup.find_all("a", class_="sister")

In [ ]:
soup.find_all("a")
soup("a")

---
## Prosty, praktyczny scraper

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


url = f'https://pl.wikipedia.org/wiki/Dane_statystyczne_o_miastach_w_Polsce'
    
page = requests.get(url)
print (page)


In [ ]:
page.content[:300]

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page.content)

In [ ]:
tables = soup.find_all('table')
len(tables)

In [ ]:
table = tables[0]
rows = table.find_all('tr')
for row in rows[:2]:
    print (row)

In [ ]:
table = tables[0]
rows = table.find_all('tr')
for row in rows[:3]:
    cells = row.find_all('td')
    for cell in cells[:4]:
        print(cell)
    print()

In [ ]:
table = tables[0]
rows = table.find_all('tr')
for row in rows[:3]:
    cells = row.find_all('td')
    for cell in cells[:4]:
        link=cell.find('a')
        if link and link!=-1:
            print(link)

In [ ]:
table = tables[0]
rows = table.find_all('tr')
for row in rows[:3]:
    cells = row.find_all('td')
    for cell in cells[:4]:
        link=cell.find('a')
        if link and link!=-1:
            print(link.text)

In [ ]:
table = tables[0]
rows = table.find_all('tr')
for row in rows[:3]:
    cells = row.find_all('td')
    for cell in cells[:4]:
        link=cell.find('a')
        if link and link!=-1:
            pass
        else:
            print(cell.text)

In [ ]:
data = []
table = tables[0]
rows = table.find_all('tr')
for row in rows:
    data_row = []
    cells = row.find_all('td')
    for cell in cells:
        link=cell.find('a')
        if link and link!=-1:
            data_row.append(link.text.strip())
        else:
            data_row.append(cell.text.strip())
    if data_row:
        data.append(data_row)
data[:4]

In [ ]:
import pandas as pd

df = pd.DataFrame(data)
df

In [ ]:
rows = table.find_all('tr')
for row in rows:
    data_row = []
    cells = row.find_all('th')
    for cell in cells:
        print(cell)
    break

In [ ]:
header = []
rows = table.find_all('tr')
for row in rows:
    data_row = []
    cells = row.find_all('th')
    for cell in cells:
        link=cell.find('a')
        if link and link!=-1:
            header.append(link.text.strip())
        else:
            header.append(cell.text.strip())
print(header)

In [ ]:
df.columns = header
df

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

cities = pd.DataFrame((pd.read_html(str( BeautifulSoup(
              requests.get('https://pl.wikipedia.org/wiki/Dane_statystyczne_o_miastach_w_Polsce').content
             ).table)))[0])
cities

In [ ]:
requests.get('https://pl.wikipedia.org/wiki/Dane_statystyczne_o_miastach_w_Polsce').content[:500]

In [ ]:
str(BeautifulSoup(
              requests.get('https://pl.wikipedia.org/wiki/Dane_statystyczne_o_miastach_w_Polsce').content
             ).table)[:1000]

In [ ]:
pd.read_html(
              str(BeautifulSoup(
              requests.get('https://pl.wikipedia.org/wiki/Dane_statystyczne_o_miastach_w_Polsce').content
             ).table)
)

In [ ]:
pd.DataFrame(
    pd.read_html(
                 str(BeautifulSoup(
                         requests.get('https://pl.wikipedia.org/wiki/Dane_statystyczne_o_miastach_w_Polsce').content
                 ).table)
    )[0]
)